In [1]:
using PlotlyJS

In [3]:
using Astrodynamics

## Timescale Conversions

In [5]:
t = Epoch(UTC, now())

2016-06-23T14:56:02.000 UTC

In [6]:
TAIEpoch(t)

2016-06-23T14:56:38.000 TAI

In [7]:
abstract HelgeTime <: UTC

In [9]:
Base.convert(::Type{Epoch{HelgeTime}}, ep::Epoch) = Epoch(HelgeTime, ep.jd, ep.jd1)+EpochDelta(seconds=300)
Epoch(HelgeTime, t)

2016-06-23T15:01:02.000 HelgeTime

### How does this work?

In [7]:
function neighbors(t, root=Any)
    super = supertype(t)
    if super == root
        return subtypes(t)
    else
        return push!(subtypes(t), super)
    end
end

function findpath(origin, target, root=Any)
    queue = [origin]
    links = Dict{DataType, DataType}()
    while !isempty(queue)
        node = shift!(queue)
        if node == target
            break
        end
        for n in neighbors(node, root)
            # Handle parametric types like IAU{Earth}
            if target <: n && isempty(subtypes(n))
                n = target
            end
            if !haskey(links, n)
                push!(queue, n)
                merge!(links, Dict{DataType, DataType}(n=>node))
            end
        end
    end
    if !haskey(links, target)
        error("No conversion path '$origin' -> '$target' found.")
    end
    path = [target]
    node = target
    while node != origin
        push!(path, links[node])
        node = links[node]
    end
    return reverse(path)
end

@generated function convert{T<:Timescale,S<:Timescale}(::Type{Epoch{T}}, obj::Epoch{S})
    ex = :(obj)
    path = findpath(S, T, Timescale)
    if length(path) == 2
        error("Please provide a method Base.convert(::Type{Astrodynamics.Epoch{$T}}, ::Astrodynamics.Epoch{$S}).")
    end
    for t in path[2:end]
        ex = :(convert(Epoch{$t}, $ex))
    end
    return :($ex)
end

convert (generic function with 1 method)

## State Vector Conversions

In [10]:
r = [
6068279.27,
-1692843.94,
-2516619.18,
]/1000

v = [
-660.415582,
5495.938726,
-5303.093233,
]/1000

t = Epoch(UTC, "2016-05-30T12:00:00.000")

iss = State(t, r, v)

State{GCRF,UTC,Earth}:
 Epoch: 2016-05-30T12:00:00.000 UTC
 Frame: GCRF
 Body:  Earth

 x: 6068.27927
 y: -1692.84394
 z: -2516.61918
 u: -0.660415582
 v: 5.495938726
 w: -5.303093232999999

 a: 6785.028010402155
 e: 0.0006796598425164399
 i: 51.698121020902995
 ω: 146.0217323119771
 Ω: 130.63072556485514
 ν: 77.57963290064639

In [14]:
State(iss, body=Mars, frame=IAU{Mars}, timescale=HelgeTime)

State{IAU{Mars},HelgeTime,Mars}:
 Epoch: 2016-05-30T12:10:00.000 HelgeTime
 Frame: IAU{Mars}
 Body:  Mars

 x: 1.7487222170069892e7
 y: 7.152981432775074e7
 z: 1.5657760507860165e7
 u: 5076.173883760601
 v: -1238.169378911845
 w: -4.00084244230769

 a: -0.0015687690931150378
 e: 4.798837581026733e10
 i: 167.99553565515913
 ω: 166.4985292021129
 Ω: 90.21093600719439
 ν: 0.020321140044021094

## Kepler Propagation

In [15]:
iss_trajectory = trajectory(iss, Kepler());

In [16]:
plot(iss_trajectory)

MethodError: MethodError: `fd` has no method matching fd(::Base64EncodePipe)

## ODE Propagation

In [17]:
iss_spherical = trajectory(iss, ODE());

In [18]:
iss_j2 = trajectory(iss, EpochDelta(days=7), ODE(gravity=J2Gravity(Earth)));

In [19]:
plot(iss_j2.trajectory)

MethodError: MethodError: `fd` has no method matching fd(::Base64EncodePipe)

In [22]:
propagator = ODE(
gravity=J2Gravity(Earth),
discontinuities=[Discontinuity(PericenterEvent(), ImpulsiveManeuver(along=-1))]
)
iss_boost = trajectory(iss, SEC_PER_DAY, propagator);

LoadError: LoadError: Impact detected.
t=4719.506182430057.
while loading In[22], in expression starting on line 5

In [21]:
plot(iss_boost.trajectory)

MethodError: MethodError: `fd` has no method matching fd(::Base64EncodePipe)